#  Nibelungenlied and Völsunga saga

In [1]:
import norsecorpus.reader as ncr

In [2]:
available_texts = ncr.get_available_texts()

In [3]:
volsunga_saga = ncr.read_tei_words(available_texts["volsunga.xml"])

In [4]:
" ".join(volsunga_saga[0][0][0])

'Hér hefr upp ok segir frá þeim manni , er Sigi er nefndr ok kallaðr , at héti sonr Óðins .'

In [5]:
all_words = [word for chapter in volsunga_saga for para in chapter for sentence in para for word in sentence]

In [6]:
set_words = set(all_words)

In [7]:
capital_words = {word for word in set_words if word and word[0].isupper()}

In [8]:
len(capital_words)

630

In [9]:
# capital_words

In [10]:
from cltk.stop.old_norse.stops import STOPS_LIST

In [11]:
# {word for word in capital_words if word.lower() not in STOPS_LIST}

-----------------------------------

## Augburg's corpus

In [12]:
import sig.nib_augsburg.nib_reader as nibaugr

In [13]:
nibaugr.MAIN_LINKS[0]

'https://www.hs-augsburg.de/~harsch/germanica/Chronologie/12Jh/Nibelungen/nib_c_00.html'

In [14]:
nibaugr.read_tei(nibaugr.MAIN_LINKS[0])[0][0]

['UNS IST> In alten', 'mæren wnders vil geseit']